In [1]:
import utils_data as ut
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizer
tokenizerBERT = BertTokenizer.from_pretrained('pranav-s/MaterialsBERT', model_max_length=512)
modelBERT = BertForMaskedLM.from_pretrained('pranav-s/MaterialsBERT')

In [4]:
classes = {'POLYMER': 1,
           'ORGANIC': 2,
           'MONOMER': 3,
           'PROP_NAME': 4,
           'INORGANIC': 5,
           'MATERIAL_AMOUNT': 6,
           'POLYMER_FAMILY': 7,
           'PROP_VALUE': 8,
           'O': 0}
max_length = 512
batch_size = 10
class NERBERTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = modelBERT.base_model
        self.linear = nn.Linear(768, len(classes) + 1)
        
    def forward(self, token):
        encoder_output= self.bert(token)
        linear_output = self.linear(encoder_output.last_hidden_state[0,:,:])
        class_output = F.softmax(linear_output, dim=1)
        return class_output

In [ ]:
model = NERBERTModel()

In [2]:
data_list = ut.read_data('train.json')

In [15]:
token_tensors_all_list[460]

tensor([[    1,     1,  1927, 16745,     1,     1,    13,    12,     1,    13,
          1982,  3845,  2193,  8862,  4860,    12,     1,    16,     1,    71,
          1930,     1,    71,    13,    16,  3824,    16,  7111,    16,  1930,
          5821,  2860,    18,     1,  2555,  2778,  2258,  3845,  1920,  3604,
          1927,  2303,     1,  1958,     1,    12,     1,    16,     1,  1930,
             1,    13,    16, 14119,  1977, 10974,  2810,  1942,  2496,  3959,
            17,  4927, 20596,    18,     1,  1930,  5056, 15422, 20364,  4146,
          2019,  3259,    89,     1,  4504,  1988,  2228,  9350,  1922,  4860,
          2032,  3216,  1958, 14119,     1,  2785,  2452,  1942, 26283,     1,
            18,     1,    16,     1,  4985,  1956,     1,  3751,     9, 14119,
          9826,    34, 22533,  7104,  8137,  1958,  2321,     1,    18,     1,
         23277,  4146,  2037,  3772,    17,  9310, 23277,  3235,  6645,  1927,
          1920,  9350,  1922,     1, 12506,  1958,  

In [14]:
token_tensors_all_list = [ut.list2token(tokenizerBERT, d['words'], max_length) for d in data_list]
token_tensors_all = torch.stack(token_tensors_all_list, dim=1)
token_tensors_all.shape

RuntimeError: stack expects each tensor to be equal size, but got [1, 512] at entry 0 and [1, 829] at entry 460

In [11]:
token_tensor = ut.list2token(tokenizerBERT, data['words'], max_length)
token_tensor.shape

torch.Size([1, 512])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 1
train_losses = []
for epoch in range(epochs):
    for data in data_list[:100]:
        token = ut.list2token(tokenizerBERT, data['words'], max_length)
        prediction = model(token)
        target = torch.tensor(ut.cat2digit(classes, data['ner'], max_length))
        loss = criterion(prediction, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_losses.append(loss)
    print(f'epoch: {epoch:2}  loss: {loss.item():10.8f}')